In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [2]:
bodyLotion_url = "https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=1000001000300140001&fltDispCatNo=&prdSort=02&pageIdx=1&rowsPerPage=24&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat1000001000300140001_Small"
bodyCream_url = "https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=1000001000300140002&fltDispCatNo=&prdSort=02&pageIdx=1&rowsPerPage=24&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=&bShowCnt=&cShowCnt=&trackingCd=Cat1000001000300140002_Small"
bodyOil_url = "https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=1000001000300140003&fltDispCatNo=&prdSort=02&pageIdx=1&rowsPerPage=24&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=&bShowCnt=&cShowCnt=&trackingCd=Cat1000001000300140003_Small"
handCream_url = "https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=1000001000300160002&fltDispCatNo=&prdSort=02&pageIdx=1&rowsPerPage=24&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=&bShowCnt=&cShowCnt=&trackingCd=Cat1000001000300160002_Small"
lipCare_url = "https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100030008&fltDispCatNo=&prdSort=02&pageIdx=1&rowsPerPage=24&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=1&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat100000100030008_Small"
skin_care="https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=1000001000800130001&fltDispCatNo=&prdSort=01&pageIdx=1&rowsPerPage=24&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat1000001000800130001_Small"

In [3]:
def crawling(url, dim_list):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)

     # 해당 카테고리 결과 딕셔너리
    res_dict = {
        "customer_id":[],
        "product_name":[],
        "ingredient":[],
        "review":[],
        "rating":[],
        "purchase_date":[]
    }
    
    dim_cnt = len(dim_list)
    for idx in range(dim_cnt):
        res_dict[dim_list[idx]] = []

    driver.implicitly_wait(2)
    prd_left_num =  int(driver.find_element(By.XPATH,'//*[@class="cate_info_tx"]/span').text)  # 해당 카테고리 총 상품 개수
    prd_page_idx = 1 
    dict_key = False #결과 딕셔너리 key 완성 여부

    while True:
        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME,'prd_info')))
        prd_cnt =  len(driver.find_elements(By.CLASS_NAME,'prd_info')) #현재 페이지에 노출된 상품 개수

        for i in range(prd_cnt):
            is_dim_ok = True
            
            try:
                wait = WebDriverWait(driver, 10)
                element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME,'prd_info')))
                driver.find_elements(By.CLASS_NAME,'prd_info')[i].click() #  j번째 상품 클릭
            except:
                print("----------------------------------------------------------------------0번 예외 처리")
                print('prdnum: '+str(i))
                prd_left_num -=1
                time.sleep(1)  
                continue
            
            try:
                time.sleep(2)
                wait = WebDriverWait(driver, 10)
                element = wait.until(EC.element_to_be_clickable((By.ID,'buyInfo')))
                driver.find_element(By.ID,'buyInfo').click() # 구매정보 버튼 클릭
                time.sleep(4)
                prd_igd =driver.find_element(By.XPATH,'//*[@id="artcInfo"]/dl[7]/dd').text # 성분 저장
            except:
                print("----------------------------------------------------------------------성분------ 예외 처리")
                print('prdnum: '+str(i))
                prd_left_num -=1
                driver.back()
                time.sleep(1)  
                continue 
                

            try:
                driver.find_element(By.CLASS_NAME,'goods_reputation').click() # 리뷰 버튼 클릭
                wait = WebDriverWait(driver, 3)
                element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME,'info_user')))

                for idx in range(dim_cnt):
                    if(dim_list[idx] != driver.find_element(By.XPATH,'//*[@id="gdasList"]/li[1]/div[2]/div[2]/dl['+str(idx+1)+']/dt/span').text):
                        is_dim_ok =False
                        break

                if is_dim_ok == False:
                    print("----------------------------------------------------------------------차원이 안맞음")
                    print('prdnum: '+str(i))
                    prd_left_num -=1
                    driver.back()
                    time.sleep(1)  
                    continue

                prd_name = driver.find_element(By.CLASS_NAME,'prd_name').text # 상품명
                review_left_num = int(driver.find_element(By.XPATH,'/html/body/div[3]/div[9]/div/ul/li[3]/a/span').text.replace(',',"").replace('(',"").replace(')',""))  # 총 리뷰 개수
                review_page_idx = 1
            except:
                print("----------------------------------------------------------------------1번 예외 처리")
                print('prdnum: '+str(i))
                prd_left_num -=1
                driver.back()
                time.sleep(1)  
                continue

            while True:
                time.sleep(1)
                review_cnt = len(driver.find_elements(By.XPATH,'//*[@id="gdasList"]/li')) #현재 페이지에 노출된 리뷰 개수
                print("prdNum:"+str(i)+'review_page_idx:'+ str(review_page_idx))
                for j in range(review_cnt):
                    try:
                        for k in range(dim_cnt):
                            res_dict[dim_list[k]].append(driver.find_element(By.XPATH,"//*[@id='gdasList']/li["+str(j+1)+"]/div[2]/div[2]/dl["+str(k+1)+"]/dd/span").text)             
                        res_dict['customer_id'].append(driver.find_element(By.XPATH,"//*[@id='gdasList']/li["+str(j+1)+"]/div[1]/div/p[1]/a[1]").text)
                        res_dict['rating'].append(driver.find_element(By.XPATH,"//*[@id='gdasList']/li["+str(j+1)+"]/div[2]/div[1]/span[1]/span").text[-2]) 
                        res_dict['purchase_date'].append(driver.find_element(By.XPATH,"//*[@id='gdasList']/li["+str(j+1)+"]/div[2]/div[1]/span[2]").text) 
                        res_dict['review'].append(driver.find_element(By.XPATH,"//*[@id='gdasList']/li["+str(j+1)+"]/div[2]/div[3]").text.replace('\n',"")) 
                        res_dict['product_name'].append(prd_name) 
                        res_dict['ingredient'].append(prd_igd) 
                    except:
                        print('===========2번 예외 처리')
                        print(str(prd_name) + "예외 발생:"+" review_page_idx:, "+ str(review_page_idx) + "j: " + str(j))
                        for key in res_dict.keys():
                            if(len(res_dict[key])>len(res_dict['product_name'])):
                                res_dict[key].pop()
                                print(len(res_dict[key]))    
                    finally:
                        review_left_num -= 1

                # 남은 리뷰 없으면 종료, 있으면 다음 리뷰 페이지
                if (review_left_num) <=0 or (review_page_idx==100):
                    break
                else:
                    try:
                        review_page_remainder = review_page_idx%10
                        review_page_idx += 1

                        if review_page_remainder == 0:
                            wait = WebDriverWait(driver, 10)
                            element = wait.until(EC.element_to_be_clickable((By.XPATH,"//*[@class='pageing']//*[@class='next']")))
                            driver.find_element(By.XPATH, "//*[@class='pageing']//*[@class='next']").click()
                            time.sleep(1)
                        elif review_page_idx<=10:
                            wait = WebDriverWait(driver, 10)
                            element = wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@class="pageing"]/a['+str(review_page_remainder)+']')))
                            driver.find_element(By.XPATH,'//*[@class="pageing"]/a['+str(review_page_remainder)+']').click()
                            time.sleep(1)
                        else:
                            wait = WebDriverWait(driver, 10)
                            element = wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@class="pageing"]/a['+str(review_page_remainder+1)+']')))
                            driver.find_element(By.XPATH,'//*[@class="pageing"]/a['+str(review_page_remainder+1)+']').click()
                            time.sleep(1)
                    except:
                        print(review_left_num)
                        print("==============================================================3번째 예외?????")
                        break

            prd_left_num -=1
            driver.back()
            time.sleep(1)   
            print('=====================================================prd_left_num 확인')
            print(prd_left_num)
            if (prd_left_num == 0):
                break   
        if (prd_left_num <= 0):
            return res_dict
            break   
        try:   
            prd_page_remainder = prd_page_idx%10
            prd_page_idx += 1

            if prd_page_remainder == 0:
                wait = WebDriverWait(driver, 10)
                element = wait.until(EC.element_to_be_clickable((By.XPATH,"//*[@class='pageing']//*[@class='next']")))
                driver.find_element(By.XPATH, "//*[@class='pageing']//*[@class='next']").click()
                time.sleep(1)
            elif prd_page_idx<=10:
                wait = WebDriverWait(driver, 10)
                element = wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@class="pageing"]/a['+str(prd_page_remainder)+']')))
                driver.find_element(By.XPATH,'//*[@class="pageing"]/a['+str(prd_page_remainder)+']').click()
                time.sleep(1)
            else:
                wait = WebDriverWait(driver, 10)
                element = wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@class="pageing"]/a['+str(prd_page_remainder+1)+']')))
                driver.find_element(By.XPATH,'//*[@class="pageing"]/a['+str(prd_page_remainder+1)+']').click()
                time.sleep(1)
        except:
            print("===========================================================================4번 예외 처리")
            print("prd_page_idx:",prd_page_idx)
            print("prd_left_num:",prd_left_num)


In [ ]:
bodyLotion_dict = crawling(bodyLotion_url, ['보습력', '끈적임', '향'])

In [ ]:
bodyLotion_df = pd.DataFrame(bodyLotion_dict)

In [ ]:
bodyLotion_df.to_csv('bodyLotion_data.csv', index=False)

In [5]:
bodyLotion_data = pd.read_csv('bodyLotion_data.csv')

In [6]:
bodyLotion_data

,customer_id,product_name,ingredient,review,rating,purchase_date,보습력,끈적임,향
0,주ya,더마비 데일리 모이스처 바디로션 500ml 기획 (500ml+80ml),"정제수, 글리세린, 부틸렌글라이콜, 카프릴릭/카프릭 트리글리세라이드, 세틸에칠헥사노...","올리브영 어워드 상위 랭크, 화해에서 검증 된 제품이라 신뢰가 가서 구입한 더마비 ...",5,2021.01.17,보습이 뛰어나요,끈적임이 없어요,아주 만족해요
1,도움요정,더마비 데일리 모이스처 바디로션 500ml 기획 (500ml+80ml),"정제수, 글리세린, 부틸렌글라이콜, 카프릴릭/카프릭 트리글리세라이드, 세틸에칠헥사노...",🌸반갑습니다~ 행복한 올리브영입니다🤗🌸안녕하세요! 올리브영에서 도움요정으로 근무중인...,5,2022.09.03,보습이 뛰어나요,끈적임이 없어요,아주 만족해요
2,한걸음,더마비 데일리 모이스처 바디로션 500ml 기획 (500ml+80ml),"정제수, 글리세린, 부틸렌글라이콜, 카프릴릭/카프릭 트리글리세라이드, 세틸에칠헥사노...",저렴하게 구입한지는 모르겠지만 집에 바디로션이 다 떨어져서 급하게 찾아보다가 전에 ...,4,2022.08.30,보습이 뛰어나요,보통이에요,보통이에요
3,도움요정,더마비 데일리 모이스처 바디로션 500ml 기획 (500ml+80ml),"정제수, 글리세린, 부틸렌글라이콜, 카프릴릭/카프릭 트리글리세라이드, 세틸에칠헥사노...",🌸반갑습니다~ 행복한 올리브영입니다🤗🌸안녕하세요! 올리브영에서 도움요정으로 근무중인...,5,2022.07.31,보통이에요,끈적임이 없어요,보통이에요
4,leciellevent,더마비 데일리 모이스처 바디로션 500ml 기획 (500ml+80ml),"정제수, 글리세린, 부틸렌글라이콜, 카프릴릭/카프릭 트리글리세라이드, 세틸에칠헥사노...","데일리로 부담없이 듬뿍듬뿍 사용하기 좋은 바디로션으로, 한 여름과 한 겨울을 제외한...",5,2021.02.25,보습이 뛰어나요,보통이에요,보통이에요
...,...,...,...,...,...,...,...,...,...
71234,dbthdd****,존슨즈베이비 핑크로션 500ml,정제수미네랄오일프로필렌글라이콜글라이콜스테아레이트디메치콘글리세린폴리소르베이트20하이드...,향도 좋고 발림성도 좋아요 가격도 마음에 드네요,5,2020.09.30,보습이 뛰어나요,끈적임이 없어요,아주 만족해요
71235,hhh****,존슨즈베이비 핑크로션 500ml,정제수미네랄오일프로필렌글라이콜글라이콜스테아레이트디메치콘글리세린폴리소르베이트20하이드...,향도 순하고 촉촉하고 좋아요ㅋㅋㅋ다음에 또ㅠ구입할래여,5,2020.09.28,보통이에요,보통이에요,보통이에요
71236,choigo****,존슨즈베이비 핑크로션 500ml,정제수미네랄오일프로필렌글라이콜글라이콜스테아레이트디메치콘글리세린폴리소르베이트20하이드...,좋아요 계속재구매중 좋아요 계속재구매중좋아요,5,2020.07.19,보습이 뛰어나요,끈적임이 없어요,아주 만족해요
71237,tigerbro****,존슨즈베이비 핑크로션 500ml,정제수미네랄오일프로필렌글라이콜글라이콜스테아레이트디메치콘글리세린폴리소르베이트20하이드...,NaN,5,2020.05.13,보습이 뛰어나요,끈적임이 없어요,아주 만족해요
